Overall procedure to suggest patient's symptoms:

1. get [ patient's gender, patient's age, and a patient's selected symptom ]
2. filtered out symptoms by gender
3. select only proper range of patient's age 
    (ex, if the patient was 25, the candidates for using in prediction could be between 25-30)
4. after filtererd out the outrange data, using the language model to calculate a proper suggestion

Import libraries and data

In [36]:
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from collections import Counter

import pickle

data_path = "symptom_data.csv"
df = pd.read_csv(data_path, encoding="utf-8")
df = df.drop(columns=['summary'])

df['feature'] = df['gender'] + ', ' + df['age'].astype(str) + ', ' + df['search_term']

cv=CountVectorizer(max_features=1000)
vector=cv.fit_transform(df['feature'].values.astype('U')).toarray()
similarity=cosine_similarity(vector)

def recommend(patient_data):
    symptom_list = []
    matching_data = df[df['feature'] == patient_data.get('gender') + ', ' + str(patient_data.get('age')) + ', '+ patient_data.get('search_term')]
    if matching_data.empty:
        print('no symptom matched')
        
    else :
        index = matching_data.index[0]

        distance = sorted(list(enumerate(similarity[index])), reverse=True, key=lambda vector:vector[1])
        possible_solution = []
        for i in distance:
            possible_solution.append(df.iloc[i[0]].search_term.strip().split(', '))
            flatted_solutions = [item for sublist in possible_solution for item in sublist]
            solution_counts = Counter(flatted_solutions)
            sorted_solutions_by_frequency = sorted(solution_counts.items(), key=lambda x: x[1], reverse=True)
        for i in sorted_solutions_by_frequency[0:5]:
            symptom_list.append(i[0])

    return symptom_list

pickle.dump(df, open('patient_data.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))